Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [9]:
from random import choices,random,randint,sample
from copy import copy
import numpy as np
import lab9_lib
from pprint import pprint


## PARAMETERS INITIALIZATION & PROBLEM GENERATION

In [10]:
PROBLEM_DIM = 1

GENES_PER_LOCUS = 1
GENOME_LENGHT=1000*GENES_PER_LOCUS
POP_DIM= 1000
OFFSPRING_SIZE = 600
N_GENERATIONS = 1000
N_GENS_WO_IMPROVEMENT_EXTINCTION = 10
MUTATION_PROB = 0.2

fitness = lab9_lib.make_problem(PROBLEM_DIM)

# for n in range(10):
#     ind = choices([0, 1], k=50)
#     print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

# print(fitness.calls)

## INDIVIDUAL CLASS

In [11]:
def hamming_distance(g1, g2):
    return sum([1 for i in range(GENOME_LENGHT) if g1[i] != g2[i]])

In [12]:
class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)
    
    def get_genome(self):
        return self.genome

    def get_fitness(self):
        return self.fitness

    def update_fitness(self,upd_fit):
        self.fitness = upd_fit 

    def set_genome_update_fitness(self, genome):
        self.genome = genome
        self.fitness = fitness(genome)

## GA

In [13]:
def one_cut_xover(g1, g2):
    cut = randint(0, GENOME_LENGHT)
    joined = g1.get_genome()[:cut] + g2.get_genome()[cut:]
    return Individual(joined)


def two_cut_xover(g1, g2):
    cut1 = randint(0, GENOME_LENGHT)
    cut2 = randint(0, GENOME_LENGHT)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    joined = (
        g1.get_genome()[:cut1] + g2.get_genome()[cut1:cut2] + g1.get_genome()[cut2:]
    )
    return Individual(joined)


def tournament_selection(pop, size=10):
    c_ind = choices(range(len(pop)), k=size)
    selected_individuals = [pop[i] for i in c_ind]
    # print("sel", selected_individuals)
    return sorted(selected_individuals, key=lambda i: i.get_fitness(), reverse=True)[0]


def mutation(g):
    # Bit Flip Mutation
    mut_prob=1/GENOME_LENGHT
    genome = g.get_genome()
    old_genome=g.get_genome()
    for i in range(GENOME_LENGHT):
        if random() < mut_prob:
            genome[i] = 1 - genome[i]
    return Individual(genome)


def extinction(pop,to_keep=0):
    to_remove = np.random.choice(pop, size=(POP_DIM-to_keep) , replace=False)
    for i in to_remove:
        pop.remove(i)
    pop += [Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM-to_keep)]
    return pop





def two_level_diversity_selection(pop, n_best):
    # first select individuals respect to the fitness value,
    # then choose for reproduction the most different individuals
    selected_individuals = sample(range(len(pop)),k=30)  #extract 20 indexes of individuals from pop
    selected_individuals = [pop[i] for i in selected_individuals] #get the individuals from pop

    to_compare = sorted(selected_individuals, key=lambda i: i.get_fitness(), reverse=True)[:n_best] #get the best fitness wise

    max_distance = -1
    selected = []
    for i in range(len(to_compare)):
        for j in range(i + 1, len(to_compare)):
            distance = hamming_distance(to_compare[i].get_genome(), to_compare[j].get_genome())
            if distance > max_distance:
                max_distance = distance
                selected = [to_compare[i], to_compare[j]]
    # print("max_distance", max_distance)
    return selected[0],selected[1]

 

In [14]:


def compute_pop_entropy(pop):
    # compute population entropy
    # to avod numerical problems, we add a small value (1e-10) to the probability
    concat=[pop[i].get_genome() for i in range(len(pop))][0]
    bincount=np.bincount(concat)
    bin_prob=bincount / len(concat)
    entropy = -sum([p * np.log2(p+1e-10) for p in bin_prob ])
    return entropy

def entropy_stocastic_selection(pop):
    max_entropy = -1
    candidates = sample(range(len(pop)), k=30)
    candidates = [pop[i] for i in candidates]
    candidates=sorted(candidates, key=lambda i: i.get_fitness(), reverse=True)[:3]
    for i in range(len(candidates)):
        for j in range(i + 1, len(candidates)):
            copy_pop = copy(pop)
            new_ind=two_cut_xover(candidates[i], candidates[j]) # crossover
            copy_pop.append(new_ind)
            new_etr=compute_pop_entropy(copy_pop)

            if new_etr > max_entropy:
                max_entropy = new_etr
                best_ind = new_ind
    return best_ind


In [15]:
def apply_fitness_penalty(pop,off):
    # apply a penalty to the fitness of the individuals
    # this is done to avoid that all individuals have the same fitness
    radius=10
    alpha=1.5 #alpha>0
    beta=2.5  #beta>1
    pop=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)
    off_ind=pop.index(off)
    if off_ind<radius:
        min_ind=0
    else:
        min_ind=off_ind-radius
    if off_ind+radius>len(pop)-1:
        max_ind=len(pop)
    else:
        max_ind=off_ind+radius
    for i in range(min_ind,max_ind):
       dist=hamming_distance(pop[i].get_genome(),off.get_genome())
       if dist<10:
            pen=(1-(pop[i].get_fitness()/radius)**alpha)
            new_fitness=(pop[i].get_fitness()**beta)/pen
    return pop

In [16]:
# counter used to triger extinction
countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

# initial population
pop=[Individual(choices([0, 1], k=GENOME_LENGHT)) for _ in range(POP_DIM)]


best=max(pop, key=lambda i: i.get_fitness())

for gen in range(N_GENERATIONS):

    entropy_gen=compute_pop_entropy(pop)

    if countdown_to_extinction == 0:
        print("Extinction!")
        to_keep=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:10]
        pop=to_keep+extinction(pop[10:],to_keep=10) 

    # if entropy_gen<0.65:
    #     to_keep=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:500]
    #     pop=to_keep+extinction(pop[500:],to_keep=500) 
    #     print("PP",len(pop))

    for off in range(OFFSPRING_SIZE):

        if entropy_gen < MUTATION_PROB:
            p=tournament_selection(pop)
            offspring = mutation(p)
        else:
            p1,p2 = two_level_diversity_selection(pop, 3)
            offspring = two_cut_xover(p1, p2)
            # offspring = entropy_stocastic_selection(pop)
        
        pop.append(offspring)
        pop=apply_fitness_penalty(pop,offspring)
    pop=sorted(pop, key=lambda i: i.get_fitness(), reverse=True)[:POP_DIM]


    if pop[0].get_fitness() == best.get_fitness():
        countdown_to_extinction-=1
    else:
        countdown_to_extinction = N_GENS_WO_IMPROVEMENT_EXTINCTION

    best=pop[0]
    
    if best.get_fitness() == 1:
        print(f"Solution found at generation {gen}:{''.join(str(g) for g in best.get_genome())}")
        break

    print(f"Generation {gen}: {best.get_fitness():.2%}, entropy: {entropy_gen}")
print(f"Best individual: {''.join(str(g) for g in best.get_genome())}")
pprint(fitness.calls)


Generation 0: 60.80%, entropy: 0.9997114414642708
Generation 1: 63.50%, entropy: 0.9660780973066864
Generation 2: 64.90%, entropy: 0.9467554493946542
Generation 3: 67.10%, entropy: 0.9349579705152063
Generation 4: 68.90%, entropy: 0.9139014132043304
Generation 5: 69.90%, entropy: 0.8943244158696739
Generation 6: 71.20%, entropy: 0.8825098585514477
Generation 7: 71.90%, entropy: 0.8661236811100869
Generation 8: 72.60%, entropy: 0.8568098048356128
Generation 9: 73.00%, entropy: 0.8471460078574488
Generation 10: 73.20%, entropy: 0.8414646359196365
Generation 11: 73.40%, entropy: 0.8385800997069459
Generation 12: 73.40%, entropy: 0.8356661469433135
Generation 13: 73.40%, entropy: 0.8356661469433135
Generation 14: 73.40%, entropy: 0.8356661469433135
Generation 15: 73.40%, entropy: 0.8356661469433135
Generation 16: 73.40%, entropy: 0.8356661469433135
Generation 17: 73.40%, entropy: 0.8356661469433135
Generation 18: 73.40%, entropy: 0.8356661469433135
Generation 19: 73.40%, entropy: 0.8356661

KeyboardInterrupt: 